<a href="https://colab.research.google.com/github/lcipolina/Ray_tutorials/blob/main/WandB_RLLIB_2_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ray[rllib] --quiet #ray, version 2.6.1
! ray --version

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 836.9/836.9 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.0 MB/s eta 0:00:00
ray, version 2.6.2


In [2]:
!pip install wandb --quiet
import wandb
from ray.air.integrations.wandb import WandbLoggerCallback

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.7 MB/s eta 0:00:00


In [3]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
import os
import numpy as np
import ray
#from ray.rllib.env import MultiAgentEnv #RLLIB 2.2
from ray import air, tune
from ray.rllib.algorithms.ppo import PPOConfig

#from ray.rllib.env.multi_agent_env import MultiAgentEnv #new RLLIB 2.3
#from ray.rllib.policy.policy import PolicySpec #For multi-policy mapping

/usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <
2023-08-09 09:13:07,907	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
/usr/local/lib/python3.10/dist-packages/google/rpc/__init__.py:20: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.rpc')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  pkg_resources.declare_namespace(__name__)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2349: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as

In [5]:
# New Gym on RLLIB 2.3
!pip install gymnasium --quiet
import gymnasium as gym
#from gym.spaces import MultiDiscrete
#from gym.spaces import Tuple, Box, MultiDiscrete, Discrete

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Configure experiments and train

In [6]:
from ray.rllib.examples.env.two_step_game import TwoStepGame

In [7]:
#https://github.com/ray-project/ray/blob/17596b03d189f2e6d642fb5690aefb01faec90d1/rllib/examples/centralized_critic_2.py

#Simplified version

config = PPOConfig()\
        .environment(TwoStepGame)\
        .framework("torch")\
        .rollouts(batch_mode="complete_episodes", num_rollout_workers=0)


2023-08-09 09:13:20,426	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.


In [8]:
callbacks = [WandbLoggerCallback(
	project='WANDB_PROJECT',
        log_config=True)]

### NOTE: it only works with local_mode = False!

In [9]:
if ray.is_initialized(): ray.shutdown()
ray.init(local_mode=False,include_dashboard=False, ignore_reinit_error=True)


stop = {
        "training_iteration": 1,
        "timesteps_total": 1
    }

tuner = tune.Tuner(
        "PPO",
        param_space=config.to_dict(),
        run_config=air.RunConfig(  stop=stop, verbose=1, callbacks=callbacks
                                 ),
       )

results = tuner.fit()

# Get the best result based on a particular metric.
best_result = results.get_best_result(metric="episode_reward_mean", mode="max") #TODO: en realidad, tengo que max por separado
result_df = best_result.metrics_dataframe
print("BEST RESULTS:")
print(result_df[["training_iteration",'episode_reward_mean','episode_len_mean']])

2023-08-09 09:13:27,048	INFO worker.py:1621 -- Started a local Ray instance.
2023-08-09 09:13:32,580	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
2023-08-09 09:13:32,596	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future!
2023-08-09 09:13:32,747	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
/usr/local/lib/python3.10/dist-packages/gymnasium/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lower

+--------------------------------------------------------+
| Configuration for experiment     PPO                   |
+--------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator |
| Scheduler                        FIFOScheduler         |
| Number of trials                 1                     |
+--------------------------------------------------------+

View detailed results here: /root/ray_results/PPO
To visualize your results with TensorBoard, run: `tensorboard --logdir /root/ray_results/PPO`

Trial status: 1 PENDING
Current time: 2023-08-09 09:13:33. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+----------------------------------------+
| Trial name                    status   |
+----------------------------------------+
| PPO_TwoStepGame_036a7_00000   PENDING  |
+----------------------------------------+



(pid=1429) /usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=1429)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=1429) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(pid=1429) /usr/local/lib/python3.10/dist-packages/google/rpc/__init__.py:20: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.rpc')`.
(pid=1429) Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
(pid=1429)   pkg_resources.declare_namespace(__name__)
(pid=1429) /usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2349: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
(pid=1429) Implement

Training started with configuration:
+---------------------------------------------------------------------------+
| Training config                                                           |
+---------------------------------------------------------------------------+
| _AlgorithmConfig__prior_exploration_config/type        StochasticSampling |
| _disable_action_flattening                                          False |
| _disable_execution_plan_api                                          True |
| _disable_initialize_loss_from_dummy_batch                           False |
| _disable_preprocessor_api                                           False |
| _enable_learner_api                                                  True |
| _enable_rl_module_api                                                True |
| _fake_gpus                                                          False |
| _is_atari                                                                 |
| _learner_class           

(PPO pid=1429) Install gputil for GPU system monitoring.


...

Trial status: 1 RUNNING
Current time: 2023-08-09 09:14:33. Total running time: 1min 0s
Logical resource usage: 1.0/2 CPUs, 0/0 GPUs
+----------------------------------------+
| Trial name                    status   |
+----------------------------------------+
| PPO_TwoStepGame_036a7_00000   RUNNING  |
+----------------------------------------+



(_QueueActor pid=1527) /usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(_QueueActor pid=1527)   if (distutils.version.LooseVersion(tf.__version__) <
(_QueueActor pid=1527) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(_QueueActor pid=1527) /usr/local/lib/python3.10/dist-packages/google/rpc/__init__.py:20: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.rpc')`.
(_QueueActor pid=1527) Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
(_QueueActor pid=1527)   pkg_resources.declare_namespace(__name__)
(_QueueActor pid=1527) /usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2349: DeprecationWarning: 

Training finished iteration 1 at 2023-08-09 09:15:00. Total running time: 1min 27s
+---------------------------------------------+
| Training result                             |
+---------------------------------------------+
| episodes_total                         2000 |
| num_env_steps_sampled                  4000 |
| num_env_steps_trained                     0 |
| sampler_results/episode_len_mean          2 |
| sampler_results/episode_reward_mean   4.558 |
+---------------------------------------------+

Training saved a checkpoint for iteration 1 at: /root/ray_results/PPO/PPO_TwoStepGame_036a7_00000_0_2023-08-09_09-13-33/checkpoint_000001

Training completed after 1 iterations at 2023-08-09 09:15:00. Total running time: 1min 27s

Trial status: 1 TERMINATED
Current time: 2023-08-09 09:15:00. Total running time: 1min 27s
Logical resource usage: 1.0/2 CPUs, 0/0 GPUs
+--------------------------------------------------------------------------------------------------------------------

(_WandbLoggingActor pid=1528) wandb: Waiting for W&B process to finish... (success).
(_WandbLoggingActor pid=1528) wandb: 
(_WandbLoggingActor pid=1528) wandb: Run history:
(_WandbLoggingActor pid=1528) wandb:                                              agent_timesteps_total ▁
(_WandbLoggingActor pid=1528) wandb:                                  config/_disable_action_flattening ▁
(_WandbLoggingActor pid=1528) wandb:                                 config/_disable_execution_plan_api ▁
(_WandbLoggingActor pid=1528) wandb:                   config/_disable_initialize_loss_from_dummy_batch ▁
(_WandbLoggingActor pid=1528) wandb:                                   config/_disable_preprocessor_api ▁
(_WandbLoggingActor pid=1528) wandb:                                         config/_enable_learner_api ▁
(_WandbLoggingActor pid=1528) wandb:                                       config/_enable_rl_module_api ▁
(_WandbLoggingActor pid=1528) wandb:                                                 

BEST RESULTS:
   training_iteration  episode_reward_mean  episode_len_mean
0                   1                4.558               2.0


### Other things that didn't work

In [ ]:
!from ray.rllib.env.wrappers.multi_agent_env_compatibility import MultiAgentEnvCompatibility #Don't know how this works. Aparently is passed to the config

from: can't read /var/mail/ray.rllib.env.wrappers.multi_agent_env_compatibility


In [ ]:
# Simple MARL custom Environment

'''Simple Multi-agent environment to test things
'''


class TurnEnv(MultiAgentEnv):

    def __init__(self, config=None):
        super().__init__()
        self.num_agents              = 2
        self.t                       = 0
        self.agent_lst               = list(range(self.num_agents))
        self._agent_ids              = set(self.agent_lst)

        # Now RLLIB needs both 'terminated' and 'truncated' to reset the environment
        self.terminateds             = {agent: False for agent in self.agent_lst} #new RLLIB 2.3
        self.terminateds['__all__']  = False
        self.truncateds              = {agent: False for agent in self.agent_lst} #new RLLIB 2.3
        self.truncateds['__all__']   = False
        self.info_dict               = {}

        self.observation_space       = Discrete(self.num_agents)
        self.action_space            = self.observation_space


    def reset(self, *, seed=None, options=None):
        agentID = 0
        self.info_dict = {agentID:
                     {"done": self.terminateds["__all__"]}
                     } #new RLLIB 2.3
        return {agentID: self.observation_space.sample()},self.info_dict # {agent:obs}, {agent:info}


    def step(self, action_dict):

        self.t +=1

        # TEST
        #if not action_dict:
        #   print("EMPTY ACTION DICT!!!")
        #   print('self.t =', self.t)

        if self.t == 10:
           self.truncateds             = {agent: True for agent in self.agent_lst}
           self.truncateds['__all__']  = True


        agentID = 1
        return {agentID:self.observation_space.sample()},\
               {agentID:1},\
               self.terminateds, \
               self.truncateds,\
               self.info_dict
               #obs, rewards, terminateds, truncateds, infos


In [ ]:
from gymnasium.spaces import Dict, Discrete, Tuple, MultiDiscrete
from ray.tune import register_env
from ray.tune.registry import get_trainable_cls

In [ ]:
#https://github.com/ray-project/ray/blob/master/rllib/examples/env/two_step_game.py

from ray.rllib.env.multi_agent_env import MultiAgentEnv, ENV_STATE

class A_TwoStepGame(MultiAgentEnv):
    action_space = Discrete(2)

    def __init__(self, env_config):
        super().__init__()
        self.action_space = Discrete(2)
        self.state = None
        self.agent_1 = 0
        self.agent_2 = 1
        self._skip_env_checking = True
        # MADDPG emits action logits instead of actual discrete actions
        self.actions_are_logits = env_config.get("actions_are_logits", False)
        self.one_hot_state_encoding = env_config.get("one_hot_state_encoding", False)
        self.with_state = env_config.get("separate_state_space", False)
        self._agent_ids = {0, 1}
        if not self.one_hot_state_encoding:
            self.observation_space = Discrete(6)
            self.with_state = False
        else:
            # Each agent gets the full state (one-hot encoding of which of the
            # three states are active) as input with the receiving agent's
            # ID (1 or 2) concatenated onto the end.
            if self.with_state:
                self.observation_space = Dict(
                    {
                        "obs": MultiDiscrete([2, 2, 2, 3]),
                        ENV_STATE: MultiDiscrete([2, 2, 2]),
                    }
                )
            else:
                self.observation_space = MultiDiscrete([2, 2, 2, 3])

    def reset(self, *, seed=None, options=None):
        if seed is not None:
            np.random.seed(seed)
        self.state = np.array([1, 0, 0])
        return self._obs(), {}

    def step(self, action_dict):
        if self.actions_are_logits:
            action_dict = {
                k: np.random.choice([0, 1], p=v) for k, v in action_dict.items()
            }

        state_index = np.flatnonzero(self.state)
        if state_index == 0:
            action = action_dict[self.agent_1]
            assert action in [0, 1], action
            if action == 0:
                self.state = np.array([0, 1, 0])
            else:
                self.state = np.array([0, 0, 1])
            global_rew = 0
            terminated = False
        elif state_index == 1:
            global_rew = 7
            terminated = True
        else:
            if action_dict[self.agent_1] == 0 and action_dict[self.agent_2] == 0:
                global_rew = 0
            elif action_dict[self.agent_1] == 1 and action_dict[self.agent_2] == 1:
                global_rew = 8
            else:
                global_rew = 1
            terminated = True

        rewards = {self.agent_1: global_rew / 2.0, self.agent_2: global_rew / 2.0}
        obs = self._obs()
        terminateds = {"__all__": terminated}
        truncateds = {"__all__": False}
        infos = {
            self.agent_1: {"done": terminateds["__all__"]},
            self.agent_2: {"done": terminateds["__all__"]},
        }
        return obs, rewards, terminateds, truncateds, infos

    def _obs(self):
        if self.with_state:
            return {
                self.agent_1: {"obs": self.agent_1_obs(), ENV_STATE: self.state},
                self.agent_2: {"obs": self.agent_2_obs(), ENV_STATE: self.state},
            }
        else:
            return {self.agent_1: self.agent_1_obs(), self.agent_2: self.agent_2_obs()}

    def agent_1_obs(self):
        if self.one_hot_state_encoding:
            return np.concatenate([self.state, [1]])
        else:
            return np.flatnonzero(self.state)[0]

    def agent_2_obs(self):
        if self.one_hot_state_encoding:
            return np.concatenate([self.state, [2]])
        else:
            return np.flatnonzero(self.state)[0] + 3
